In [1]:
import nbimporter
import dataset_class as StreetSign
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch import nn
from torch.optim import SGD
from sklearn.metrics import accuracy_score
from os.path import join
import torch
import numpy as np
from PIL import Image
from os import path

import matplotlib.pyplot as plt

np.random.seed(1328)
torch.random.manual_seed(1328);


C:\Users\merav\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
#resize normale
tran = transforms.Compose([StreetSign.Rescale(256),StreetSign.RandomCrop(32),StreetSign.ToTensor()])
#tran = None

datatrain = StreetSign.StreetSignDataset('DITS-full\DITS-full\DITS-detection\class\classes.csv','DITS-full\DITS-full\DITS-detection\class\image',tran)


In [3]:
#resize normale
datatest = StreetSign.StreetSignDataset('DITS-full\DITS-full\DITS-detection\detection_test\day\classes.csv','DITS-full\DITS-full\DITS-detection\detection_test\day',tran)

In [4]:
datatrain_loader = DataLoader(datatrain, batch_size=1024, shuffle=True)
#cifar100_test_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)
datatest_loader = DataLoader(datatest, batch_size=1024, shuffle=True)
#cifar100_test_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)

In [5]:
from torch import nn
from torchvision.models import squeezenet1_0

# Definisci la tua funzione get_model
def get_model(num_class=101):
    model = squeezenet1_0(pretrained=True)
    model.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1, 1), stride=(1, 1))
    model.num_classes = num_class
    return model

In [6]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from os.path import join

class AverageValueMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.sum = 0
        self.num = 0

    def add(self, value, num):
        self.sum += value * num
        self.num += num

    def value(self):
        try:
            return self.sum / self.num
        except ZeroDivisionError:
            return None

def trainval_classifier(model, train_loader, test_loader, exp_name='experiment', lr=0.01, epochs=10, momentum=0.99, logdir='logs'):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr, momentum=momentum)

    # Meters (misuratori)
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()

    # TensorBoard writer (scrittore per TensorBoard)
    writer = SummaryWriter(join(logdir, exp_name))

    # Dispositivo (CPU o GPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Definiamo un dizionario contenente i loader di training e test
    loader = {'train': train_loader, 'test': test_loader}

    # Inizializziamo il global step (passo globale)
    global_step = 0

    for e in range(epochs):
        for mode in ['train', 'test']:
            loss_meter.reset()
            acc_meter.reset()
            model.train() if mode == 'train' else model.eval()
            
            with torch.set_grad_enabled(mode == 'train'):
                for i, batch in enumerate(loader[mode]):
                    x = list(batch.values())[0].to(device).float()
                    y = list(batch.values())[1].to(device).long()  # Assicurati che y sia di tipo Long
                    y = y.squeeze()
                    
                    output=model(x)
                    
                    n = x.shape[0]  # Numero di elementi nel batch
                    global_step += n
                    l = criterion(output, y)

                    if mode == 'train':
                        l.backward()
                        optimizer.step()
                        optimizer.zero_grad()

                    acc = accuracy_score(y.to('cpu'), output.to('cpu').max(1)[1])
                    loss_meter.add(l.item(), n)
                    acc_meter.add(acc, n)

                    if mode == 'train':
                        # Logghiamo i risultati iterazione per iterazione solo durante il training
                        writer.add_scalar('loss/train', loss_meter.value(), global_step=global_step)
                        writer.add_scalar('accuracy/train', acc_meter.value(), global_step=global_step)

                # Logghiamo le stime finali dopo un'epoca (sia nel caso di training che test)
                writer.add_scalar(f'loss/{mode}', loss_meter.value(), global_step=global_step)
                writer.add_scalar(f'accuracy/{mode}', acc_meter.value(), global_step=global_step)

                # Conserviamo i pesi del modello alla fine di un ciclo di training o test
                torch.save(model.state_dict(), f'{exp_name}-{e + 1}.pth')

    return model


In [7]:
# Ottieni il modello SqueezeNet preaddestrato su Caltech-101
squeezenet_caltech101 = get_model()

# Per scopi didattici, alleniamo il modello solo per 5 epoche invece di 150
squeezenet_caltech101_finetuned = trainval_classifier( squeezenet_caltech101,datatrain_loader, 
                                  datatest_loader, 
                                  exp_name='squeezenet_caltech101_finetuning',
                                  lr=0.001, epochs=5)

C:\Users\merav\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\merav\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
datatrain_loader = DataLoader(datatrain, batch_size=1024, shuffle=True)
#cifar100_test_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)
datatest_loader = DataLoader(datatest, batch_size=1024, shuffle=True)
#cifar100_test_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)

In [9]:
# Definisci la funzione test_classifier
def test_classifier(model, loader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    predictions, labels = [], []

    for batch in loader:
        x = list(batch.values())[0].to(device).float()
        y = list(batch.values())[1].to(device).long()  
        y = y.squeeze()
        
        output = model(x)
        
        preds = output.to('cpu').max(1)[1].numpy()
        labs = y.to('cpu').numpy()
        predictions.extend(list(preds))
        labels.extend(list(labs))

    return np.array(predictions), np.array(labels)

# Utilizza la funzione per testare il modello squeezenet_caltech101_finetuned
# Assicurati di avere un dataloader di test appropriato (ad esempio, test_loader) definito in precedenza
predictions, true_labels = test_classifier(squeezenet_caltech101_finetuned, datatest_loader)

# Ora puoi utilizzare le previsioni e le etichette reali per calcolare le metriche di valutazione desiderate
# ad esempio, puoi calcolare l'accuracy così:
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


Accuracy: 0.5805471124620061


In [10]:
# Ottenere le previsioni del modello sul set di test di Caltech 101
squeezenet_caltech101_finetuned_predictions_test, caltech101_labels_test = test_classifier(
    squeezenet_caltech101_finetuned,
    datatest_loader
)

# Calcolare e stampare l'accuratezza
accuracy = accuracy_score(caltech101_labels_test, squeezenet_caltech101_finetuned_predictions_test)
print("Accuracy di Squeezenet su Caltech 101 - Ultimo modello: {:.2f}%".format(accuracy * 100))


Accuracy di Squeezenet su Caltech 101 - Ultimo modello: 58.05%
